In [1]:
%load_ext autoreload
%autoreload 2

from gpu import set_gpu
import numpy as np
import os
import adapt.utils.data_utils as prd
import adapt.loop as lp
import adapt.ml.lda as dlda
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import adapt.ml.dl_subclass as dl
import copy as cp
import tensorflow as tf
import pickle
import time
import gc as gc
import warnings
import seaborn as sns
from scipy.stats import pearsonr
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 300
%matplotlib qt
mpl.rc('font', size=12)
import matplotlib.font_manager
plt.rc('font', family='Helvetica')
set_gpu()

Num GPUs Available:  1


In [ ]:
# Count recalibrations
path = 'C:/Users/yteh/Documents/work/necal/home data/'
subs = os.listdir(path)
if 'skip' in subs:
    subs = np.delete(subs,subs.index('skip'))
bat = 32
load_mod = False
mod_tot = ['blda','lda','crlda','alda','bcnn','cnn', 'crcnn2','acnn05','acnn30','acewc30','acewc15', 'adcnn30', 'vcnn', 'bvcnn', 'avcnn', 'avcnn1', 'acnnl03','crvcnn','acewclm','crcnn','acewc00','xtra2']
ft = 'tdar'
iter = 5
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')
test_mod = 0
aug_w=0
aug_scaler=0
out_scaler=0
save_path = 'C:/Users/yteh/Documents/work/git/projects/adaptive/0404 full 70 home/'
load_path = 'C:/Users/yteh/Documents/work/git/projects/adaptive/0404 full/'
if not os.path.isdir(save_path):
    os.mkdir(save_path)

for it in range(2,iter):
    # first iteration, includes LDA; others exclude LDA
    if it == 0:
        mod_all = ['blda','lda','alda','cnn','acnn05','avcnn1']#,'crcnn']
        mod_all = ['alda','acnn05','avcnn1']
    else:
        mod_all = ['bcnn','acnn05','avcnn1','crcnn']
        

    for sub in range(7):
        print(subs[sub])
        sub_path = path + subs[sub] + '/DATA/MAT/'
        all_files = os.listdir(sub_path)
        if 'skip' in all_files:
            all_files = np.delete(all_files,all_files.index('skip'))

        # load or initialize cnn weights
        if load_mod:
            with open(load_path + subs[sub] + '_' + str(it) + '_r_accs.p','rb') as f:
                all_acc, all_recal, all_val, all_prev, all_train, all_times, all_dof,_, _, c_weights, v_weights, cl_wc, scaler_0, emg_scale, aug_w, aug_scaler,out_scaler= pickle.load(f)
        else:
            c_weights = None
            v_weights = None
            v_wc = None
            cl_wc = None
            scaler_0 = None
            all_recal = np.zeros((len(all_files),len(mod_tot)))
            all_acc = np.zeros((len(all_files),len(mod_tot)))
            all_val = np.zeros((len(all_files),len(mod_tot)))
            all_prev = np.zeros((len(all_files),len(mod_tot)))
            all_train = np.zeros((len(all_files),len(mod_tot)))
            all_times = np.zeros((len(all_files),len(mod_tot)))
            rows, cols = (len(all_files), len(mod_tot))
            all_dof = [[0]*cols]*rows

        mod_i = 0
        for mod in mod_all:
            mod_recal = np.zeros((len(all_files),))

            if 'lda' in mod:
                acc_i = 0
            else:
                acc_i = 1

            cnn = None
            ewc = None
            clda = None

            ep = 30
            recal = 0
            skip_recal = 0
            skip = False

            # Loop through files
            for i in range(0,len(all_files)):
                # Check if need to recalibrate
                if i > 0:
                    if all_acc[i,mod_tot.index(mod)] < 70:
                        skip = False
                        recal += 1
                        print('recal: ' + str(recal) + ' ' + all_files[i])
                        all_acc[i,mod_tot.index(mod)] *= -1
                        mod_recal[i] = 1
                    else:
                        skip = True
                    
                    if 'b' in mod:
                        skip = True
                        
                if not skip:
                    # load training file
                    train_file = all_files[i]
                    train_data, train_params = prd.load_caps_train(sub_path + train_file + '/traindata.mat')

                    # if first train, use two train files
                    if i == 0:
                        # load training file
                        train_file2 = all_files[i+1]
                        train_data2, train_params2 = prd.load_caps_train(sub_path + train_file2 + '/traindata.mat')
                        train_data = np.vstack((train_data,train_data2))
                        train_params = np.vstack((train_params,train_params2))
                        train_data, train_params, th = prd.threshold(train_data, train_params)
                        val_data, val_params = train_data, train_params

                        if mod == 'avcnn1':
                            if isinstance(aug_w,list) and load_mod:
                                augep = 1
                            else:
                                augep = 15
                                
                            t_label,x_clean,x_noise,x_scaled,x_n_scaled,aug_scaler,out_scaler = prd.add_noise_aug(train_data,ft='tdar')
                            aug,_ = lp.train_aug(x_scaled,x_n_scaled,t_label,ep=augep,bat=128,print_b=False)
                            del t_label,x_clean,x_noise,x_scaled,x_n_scaled
                            if augep == 1:
                                aug.set_weights(aug_w)
                            else:
                                aug_w = aug.get_weights()
                    else:
                        train_data, train_params, _ = prd.threshold(train_data, train_params,th)
                    
                        tr_i = np.zeros((train_params.shape[0],))
                        te_i = np.zeros((train_params.shape[0],))
                        for cls in np.unique(train_params[:,-1]):
                            dof = np.array(np.where(train_params[:,-1] == cls))
                            tr_i[dof[0,:dof.shape[1]//2]] = 1
                            te_i[dof[0,dof.shape[1]//2:]] = 1

                        train_temp = train_data[tr_i.astype(bool),...]
                        params_temp = train_params[tr_i.astype(bool),...]
                        val_data = train_data[te_i.astype(bool),...]
                        val_params = train_params[te_i.astype(bool),...]
                        
                        train_data, train_params = train_temp, params_temp
                        del train_temp, params_temp, tr_i, te_i
                                            
                    if i > 0:
                        # get previous dofs
                        prev_ndof = [n_dof, key, train_dof]
                        if mod[0] == 'a' or 'cr' in mod:
                            # print('prev: ' + str(train_dof))
                            # get current dofs and create key
                            train_dof = np.unique(train_params[:,-1])
                            # print('cur: ' + str(train_dof))
                            key = np.zeros((len(train_dof),))
                            # check if current dofs are all in old dof list
                            dof_ovlp = np.isin(train_dof,prev_ndof[2],assume_unique=True)
                            temp_dof = cp.deepcopy(train_dof)
                            # loop through dofs that are in previous dofs, set the keys
                            for dof in train_dof[dof_ovlp]:
                                key[train_dof==dof] = prev_ndof[1][prev_ndof[2]==dof]
                                temp_dof[train_dof==dof] = prev_ndof[2][prev_ndof[2]==dof]

                            # check if previous dofs has classes not in this set
                            dof_xtra = ~np.isin(prev_ndof[2],temp_dof,assume_unique=True)
                            temp_dof = np.hstack((temp_dof,prev_ndof[2][dof_xtra]))
                            key = np.hstack((key,prev_ndof[1][dof_xtra]))

                            # loop through dofs that are not in previous dofs (ie new classes), add keys
                            key_i = 1
                            xtra = False
                            for dof in train_dof[~dof_ovlp]:
                                # if 'lda' in mod:
                                # xtra = True
                                # key[temp_dof==dof] = np.max(key) + 1
                                    # key_i += 1
                                # else:
                                    # remove extras
                                print('removing train ' + str(dof))
                                ind = train_params[:,-1] == dof
                                train_params = train_params[~ind,...]
                                train_data = train_data[~ind,...]
                                ind = val_params[:,-1] == dof
                                val_params = val_params[~ind,...]
                                val_data = val_data[~ind,...]
                                key = np.delete(key,temp_dof==dof)
                                temp_dof = np.delete(temp_dof,temp_dof==dof)

                            train_dof = cp.deepcopy(temp_dof)
                        else:
                            # get current dofs and create key
                            train_dof = np.unique(train_params[:,-1])
                            key = np.arange(len(train_dof))
                    else:
                        # get current dofs and create key
                        train_dof = np.unique(train_params[:,-1])
                        key = np.arange(len(train_dof))

                    n_dof = len(train_dof)
                    all_dof[i][mod_tot.index(mod)] = train_dof

                    train_data, train_params = lp.check_labels(train_data,train_params,train_dof,key,False)
                    val_data, val_params = lp.check_labels(val_data,val_params,train_dof,key,False)

                    # if combining, save current training data
                    if 'cr' in mod:
                        if i > 0:
                            # combine old and new training data
                            train_data = np.vstack((train_data_0[::train_data_0.shape[0]//train_data.shape[0],...],train_data))
                            train_params = np.vstack((train_params_0[::train_params_0.shape[0]//train_params.shape[0],...],train_params))
                        ep = 15
                    
                        train_data_0 = cp.deepcopy(train_data)
                        train_params_0 = cp.deepcopy(train_params)

                    noise_b = 'lda' not in mod
                    if (mod[0] == 'a' and i > 0) or ('cr' in mod and i > 0) or (mod == 'vcnn' and i > 0):
                        _, x_clean_cnn, y_clean, _, x_train_cnn, y_train, x_train_lda, y_train_lda, _, _, _, _ = prd.prep_train_caps(train_data, train_params, emg_scale=emg_scale, scaler=scaler, num_classes=n_dof, prop_b=False, batch_size=bat, ft=ft, noise=noise_b, split=False)
                    else:
                        _, x_clean_cnn, y_clean, _, x_train_cnn, y_train, x_train_lda, y_train_lda, emg_scale, scaler, _, _ = prd.prep_train_caps(train_data, train_params, prop_b=False, batch_size=bat, ft=ft, noise=noise_b, split=False,num_classes=n_dof)
                        if ((i == 0) and (c_weights is not None)) or ((i == 0) and (v_weights is not None)):
                            scaler = cp.deepcopy(scaler_0)

                    _, _, _, _, x_val_cnn, y_val, x_val_lda, y_val_lda, _, _, _, _ = prd.prep_train_caps(val_data, val_params, emg_scale=emg_scale,scaler=scaler, prop_b=False, batch_size=bat, ft=ft, num_classes=n_dof, noise=False, split=False)

                    x_clean_scaled = scaler.inverse_transform(x_clean_cnn.reshape(x_clean_cnn.shape[0]*x_clean_cnn.shape[1],-1)).reshape(x_clean_cnn.shape)

                    del train_data, train_params, val_data, val_params

                    if 'lda' not in mod:
                        cnnlda = 'l' in mod
                        if i == 0:
                            if c_weights is None:
                                cnn, all_times[i,mod_tot.index(mod)], w_c, c_c = lp.train_models(traincnn=x_train_cnn,y_train=y_train, mod=['cnn'], n_dof=n_dof, ep=ep, cnnlda=cnnlda, bn_trainable=True,bn_training=True, prog_train=True)
                                c_weights = cp.deepcopy([cnn.enc.get_weights(),cnn.clf.get_weights()])
                                scaler_0 = cp.deepcopy(scaler)    
                            else:
                                print('setting CNN weights')
                                cnn = dl.CNN(n_class=n_dof)
                                cnn(x_train_cnn[:1,...])
                                cnn.enc.set_weights(c_weights[0])
                                cnn.clf.set_weights(c_weights[1])
                                if cnnlda:
                                    print('setting LDA weights')
                                    w_c = cp.deepcopy(cl_wc[0].astype('float32'))
                                    c_c = cp.deepcopy(cl_wc[1].astype('float32'))
                            if 'av' in mod:
                                mu_class, std_class, N = prd.set_mean(x_clean_scaled,y_clean)
                            
                            if scaler_0 is None:
                                scaler_0 = cp.deepcopy(scaler)
                            else:
                                scaler = cp.deepcopy(scaler_0)
                        else:
                            prev_w = cnn.get_weights()
                            if mod[0] == 'a':
                                if xtra:
                                    print('add neuron')
                                    cnn = dl.CNN(n_class=n_dof)
                                    cnn(x_train_cnn[:1,...])
                                    w_temp = cnn.get_weights()
                                    w_temp[:-2] = prev_w[:-2]
                                    w_temp[-2][:,:-1] = prev_w[-2]
                                    w_temp[-1][:-1] = prev_w[-1]
                                    cnn.set_weights(w_temp)
                            if 'avcnn' in mod:
                                prev_mu = [mu_class, std_class, N]
                                # generate old training data, same size as clean data
                                
                                x_gen = np.zeros(x_clean_cnn.shape)
                                num_y = x_gen.shape[0]//prev_ndof[0]
                                y_gen = np.zeros((x_gen.shape[0],n_dof))
                                y_xtra = 0
                                x_xtra = 0
                                for cl in range(mu_class.shape[0]):#prev_ndof[1].astype(int):
                                    # cl_i = np.where(prev_ndof[1]==cl)
                                    x_ind = np.squeeze(y_clean[:,cl]==1)
                                    if np.sum(x_ind) > 0:
                                        y_gen[x_ind,cl] = 1
                                        x_gen[x_ind,...] = np.random.normal(mu_class[cl,...], std_class[cl,...],x_clean_cnn[x_ind,...].shape)
                                    else:
                                        temp = np.zeros((num_y,n_dof))
                                        temp[:,cl] = 1
                                        if isinstance(y_xtra,np.ndarray):
                                            y_xtra = np.vstack((y_xtra,temp))
                                            x_xtra = np.vstack((x_xtra,np.random.normal(mu_class[cl,...], std_class[cl,...],x_clean_cnn[:num_y,...].shape)))
                                        else:
                                            y_xtra = cp.deepcopy(temp)
                                            x_xtra = np.random.normal(mu_class[cl,...], std_class[cl,...],x_clean_cnn[:num_y,...].shape)

                                if isinstance(y_xtra,np.ndarray):
                                    x_gen = np.vstack((x_gen,x_xtra))
                                    y_gen = np.vstack((y_gen,y_xtra))

                                x_gen = x_gen[np.sum(y_gen,axis=1)==1,...]
                                y_gen = y_gen[np.sum(y_gen,axis=1)==1,...]

                                if '1' in mod:
                                    start_aug = time.time()
                                    gen_noise, _, gen_noise_y = prd.aug_gen(x_gen, y_gen, aug, aug_scaler, out_scaler)
                                    print('aug: ' + str(time.time() - start_aug))
                                else:
                                    gen_noise = x_gen
                                    gen_noise_y = y_gen
                                gen_noise = scaler.transform(gen_noise.reshape(gen_noise.shape[0]*gen_noise.shape[1],-1)).reshape(gen_noise.shape)
                                
                                x_train_aug = np.vstack((gen_noise,x_train_cnn))
                                y_train_aug = np.vstack((gen_noise_y,y_train))

                                mu_class, std_class, N = prd.update_mean(x_clean_scaled,y_clean,N,mu_class,std_class,key,prev_ndof[1])
                                cnn, all_times[i,mod_tot.index(mod)],_,_ = lp.train_models(traincnn=x_train_aug,y_train=y_train_aug, mod=[cnn], n_dof=n_dof, ep=5, dec=False, lr=0.001, bn_training=False,  prog_train=False)
                            elif 'acnn' in mod: # update whole CNN and lda weights
                                ep = int(mod[-2:])
                                cnn, all_times[i,mod_tot.index(mod)], w_c, c_c = lp.train_models(traincnn=x_train_cnn,y_train=y_train, n_dof=n_dof, ep=ep, mod=[cnn], cnnlda=cnnlda, lr=0.001, bn_training=False,prog_train=False)
                            elif 'cr' in mod:
                                cnn, all_times[i,mod_tot.index(mod)], w_c, c_c = lp.train_models(traincnn=x_train_cnn,y_train=y_train, mod=[cnn], n_dof=n_dof, ep=5, cnnlda=cnnlda, bn_training=False, prog_train=False)
                            elif 'cnn' in mod: # recalibrate cnnlda
                                cnn, all_times[i,mod_tot.index(mod)], w_c, c_c = lp.train_models(traincnn=x_train_cnn,y_train=y_train, mod=['cnn'], n_dof=n_dof, ep=ep, cnnlda=cnnlda, bn_training=True, prog_train=True)

                        del test_mod
                        test_mod = dl.get_test(cnn, test_accuracy)
                        if i > 0:
                            all_prev[i,mod_tot.index(mod)] = lp.test_models(prev_x, prev_y, None, None, cnn=cnn, clda=clda, test_mod=test_mod, test_accuracy=test_accuracy)[acc_i]
                        all_val[i,mod_tot.index(mod)] = lp.test_models(x_val_cnn, y_val, None, None, cnn=cnn, clda=clda, test_mod=test_mod, test_accuracy=test_accuracy)[acc_i]
                        all_train[i,mod_tot.index(mod)] = lp.test_models(x_clean_cnn, y_clean, None, None, cnn=cnn, clda=clda, test_mod=test_mod, test_accuracy=test_accuracy)[acc_i]
                        
                        if all_val[i,mod_tot.index(mod)] < 70:
                            mod_recal[i] = -1
                            print('bad recal')
                            if i > 0:
                                n_dof, key, train_dof = prev_ndof
                                if 'vcnn' in mod:
                                    mu_class, std_class, N = prev_mu
                                cnn = dl.CNN(n_class = n_dof)
                                cnn(x_train_cnn[:1,...])
                                cnn.set_weights(prev_w)
                                del test_mod
                                test_mod = dl.get_test(cnn, test_accuracy)
                    else:
                        if i == 0:
                            N = np.zeros((n_dof),)
                            cov_class = np.zeros([x_train_lda.shape[1],x_train_lda.shape[1]])
                            mu_class = np.zeros([n_dof,x_train_lda.shape[1]])
                        prev_lda = [mu_class,cov_class,N]
                        start_time = time.time()
                        if mod[0] != 'a' or (i == 0 and mod[0] == 'a'):
                            w, c, mu_class, _, _, N, cov_class = dlda.train_lda(x_train_lda, y_train_lda, key)
                        else:
                            w, c, mu_class, cov_class, N = dlda.update_lda(x_train_lda, y_train_lda, N, mu_class, cov_class, key, prev_ndof[1])
                        all_times[i,mod_tot.index(mod)] = time.time() - start_time
                        all_val[i,mod_tot.index(mod)] = lp.test_models(None, None, x_val_lda, y_val_lda, lda=[w,c])[acc_i]
                        all_train[i,mod_tot.index(mod)] = lp.test_models(None, None, x_train_lda, y_train_lda, lda=[w,c])[acc_i]
                        if i > 0:
                            all_prev[i,mod_tot.index(mod)] = lp.test_models(None, None, prev_x_lda, prev_y_lda, lda=[w,c])[acc_i]
                        if all_val[i,mod_tot.index(mod)] < 70:
                            mod_recal[i] = -1
                            print('bad recal')
                            if i > 0:
                                mu_class, cov_class, N = prev_lda
                                n_dof, key, train_dof = prev_ndof
                        del x_train_lda, y_train_lda
                    
                    if mod_recal[i] != -1 or i == 0 :
                        prev_x = cp.deepcopy(x_val_cnn)
                        prev_y = cp.deepcopy(y_val)
                        prev_x_lda = cp.deepcopy(x_val_lda)
                        prev_y_lda = cp.deepcopy(y_val_lda)
                    
                    del  x_val_cnn, y_val, x_val_lda, y_val_lda, x_clean_cnn, y_clean,x_train_cnn, y_train
                
                if i < len(all_files)-1:
                    # load data
                    test_file = all_files[i+1]
                    test_data, test_params = prd.load_caps_train(sub_path + test_file + '/traindata.mat')
                    
                    # check class labels
                    test_data, test_params, _ = prd.threshold(test_data, test_params, th)
                    test_data, test_params = lp.check_labels(test_data,test_params,train_dof,key,True)

                    # test 
                    y_test, _, x_test_cnn, x_test_lda, y_test_lda = prd.prep_test_caps(test_data, test_params, scaler, emg_scale, num_classes=n_dof, ft=ft, split=False)
                    
                    # test 
                    if 'lda' in mod:
                        all_acc[i+1,mod_tot.index(mod)] = lp.test_models(None, None,  x_test_lda, y_test_lda, lda=[w,c])[acc_i]
                    else:
                        all_acc[i+1,mod_tot.index(mod)] = lp.test_models(x_test_cnn, y_test, x_test_lda, y_test_lda, cnn=cnn, clda=clda, test_mod=test_mod, test_accuracy=test_accuracy)[acc_i]

                    print ('Set: ' + train_file + ', Test: ' + test_file + ',', f'Accuracy: {all_acc[i+1,mod_tot.index(mod)]:.2f}', f', Val: {all_val[i,mod_tot.index(mod)]:.2f}', f', Prev: {all_prev[i,mod_tot.index(mod)]:.2f}', f', Train: {all_train[i,mod_tot.index(mod)]:.2f}')
                    del y_test, x_test_cnn, x_test_lda, y_test_lda, test_data, test_params

            all_recal[:,mod_tot.index(mod)] = mod_recal

            print('------------------------' + mod + ' ' + str(np.sum(mod_recal==1)) + ' - ' + str(np.sum(mod_recal==-1)) + ' -- ' + str(np.sum(mod_recal==-2)) + '-------------')
            mod_i += 1

            if 'cr' in mod:
            
                del train_data_0, train_params_0

        with open(save_path + subs[sub] + '_' + str(it) + '_r_accs.p','wb') as f:
            pickle.dump([all_acc, all_recal, all_val, all_prev, all_train, all_times, all_dof, mod_all, mod_tot, c_weights, v_weights, cl_wc, scaler_0, emg_scale, aug_w, aug_scaler, out_scaler],f)
        
        gc.collect(generation=2)

In [ ]:
x_s = scaler.inverse_transform(x_noise_0.reshape(x_noise_0.shape[0]*x_noise_0.shape[1],-1)).reshape(x_noise_0.shape)

ind = y_train_0[:,2] ==1
print(np.sum(ind))
fig,ax = plt.subplots(8,1,figsize=(15,10))
for i in range(8):
    ax[i].plot(np.squeeze(x_s[ind,i,...][:500,...]).T)
    ax[i].plot(np.squeeze(mu_class[0,i,...]).T)
    ax[i].set_ylim([-3,5])

In [ ]:
x_s = scaler.inverse_transform(x_clean_0.reshape(x_clean_0.shape[0]*x_clean_0.shape[1],-1)).reshape(x_clean_0.shape)

ind = y_clean_0[:,1] ==1
print(np.sum(ind))
fig,ax = plt.subplots(8,1,figsize=(15,10))
for i in range(8):
    ax[i].plot(np.squeeze(x_s[ind,i,...]).T)
    ax[i].plot(np.squeeze(mu_class[0,i,...]).T)

In [ ]:
ind = y_gen[:,0] ==1
print(np.sum(ind))
fig,ax = plt.subplots(8,1,figsize=(15,10))
for i in range(8):
    ax[i].plot(np.squeeze(x_gen[ind,i,...]).T)

In [ ]:
ind = gen_noise_y[:,2] ==1
x_s = scaler.inverse_transform(gen_noise.reshape(gen_noise.shape[0]*gen_noise.shape[1],-1)).reshape(gen_noise.shape)

fig,ax = plt.subplots(8,1,figsize=(15,10))
for i in range(8):
    ax[i].plot(np.squeeze(x_s[ind,i,...]).T)
    ax[i].set_ylim([-3,5])

In [ ]:
train_params.shape

In [ ]:
ind = gen_noise_y[:,1]==1

In [ ]:
fig,ax = plt.subplots(8,1,figsize=(15,10))
for i in range(8):
    ax[i].plot(np.squeeze(gen_noise[ind,i,...][-50:,...]).T)

In [ ]:
with open(subs[sub] + '_' + str(it) + '_r_accs.p','wb') as f:
    pickle.dump([all_acc, all_recal, all_val, all_prev, all_train, all_times, all_dof, mod_all, mod_tot, c_weights, v_weights, cl_wc, scaler_0, emg_scale, aug.get_weights(), aug_scaler],f)

In [ ]:
# Count recalibrations
path = 'C:/Users/yteh/Documents/work/necal/home data/'
subs = os.listdir(path)
if 'skip' in subs:
    subs = np.delete(subs,subs.index('skip'))
ft = 'tdar'

for sub in range(4,5):
    print(subs[sub])
    sub_path = path + subs[sub] + '/DATApre/MAT/'
    all_files = os.listdir(sub_path)
    if 'skip' in all_files:
        all_files = np.delete(all_files,all_files.index('skip'))

    train_file = all_files[0]
    train_data, train_params = prd.load_caps_train(sub_path + train_file + '/traindata.mat')
    t_label,x_clean,x_noise,x_scaled,x_n_scaled,scaler = prd.add_noise_aug(train_data,ft='tdar')

    out = lp.train_aug(x_scaled,x_n_scaled,t_label,ep=30,bat=128,print_b=False)
    aug = out[0]
    gc.collect(generation=2)

In [ ]:
_, x_clean_cnn, y_clean, _, x_train_cnn, y_train, x_train_lda, y_train_lda, _, _, _, _ = prd.prep_train_caps(train_data, train_params, emg_scale=emg_scale, scaler=scaler, num_classes=n_dof, prop_b=False, batch_size=bat, ft=ft, noise=noise_b, split=False)

In [ ]:
noisy,clean,y=aug_gen(raw, params, mod, scaler)

In [ ]:
mod = out[0]
typ = .3
test = mod(x_scaled[t_label==typ,...],t_label[t_label==typ])

In [ ]:
for i in range(test.shape[0]):
    plt.plot(test[i,:])

In [ ]:
# Count recalibrations
path = 'C:/Users/yteh/Documents/work/necal/home data/'
subs = os.listdir(path)
if 'skip' in subs:
    subs = np.delete(subs,subs.index('skip'))
bat = 32
ft = 'tdar'
iter = 1
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')
plot_mod = ['lda','alda','cnn','acnn03','avcnn','crcnn']
plot_mod = ['lda','alda','acnn05','avcnn','avcnn1','crcnn']
for plot_i in range(2):
    for sub in range(7):    
        it_acc = []
        it_recal = []
        it_fail = []
        it_val = []
        it_prev = []
        it_train = []
        it_times = []
        it_replaced = []
        for it in range(iter):#iter):
        
            # load or initialize cnn weights
            if plot_i == 1:
                # with open('0323 full run pre and post/' + subs[sub] + '_' + str(it) + '_r_accs.p','rb') as f:
                #     all_acc, all_recal, all_val, all_prev, all_train, all_times, _, mod_tot, c_weights, v_weights, cl_wc, scaler_0, emg_scale = pickle.load(f)
                with open('0403 pre/'+subs[sub] + '_' + str(it) + '_r_accs.p','rb') as f:
                    all_acc, all_recal, all_val, all_prev, all_train, all_times, all_dof, _, mod_tot, c_weights, v_weights, cl_wc, scaler_0, emg_scale,_,_,_ = pickle.load(f)
            else:
                with open('Z:/Lab Member Folders/Yuni Teh/home data/0404 full 70/'+subs[sub] + '_' + str(it) + '_r_accs.p','rb') as f:
                    all_acc, all_recal, all_val, all_prev, all_train, all_times, all_dof, _, mod_tot, c_weights, v_weights, cl_wc, scaler_0, emg_scale,_,_,_= pickle.load(f)
            
            lda_ind = mod_tot.index('alda') + 1
            all_acc[all_acc==0] = np.nan
            all_val[all_val==0] = np.nan
            all_prev[all_prev==0] = np.nan
            all_train[all_train==0] = np.nan
            all_times[all_times==0] = np.nan

            it_acc.append(all_acc)
            it_recal.append(np.sum(all_recal==1,axis=0))
            it_fail.append(np.sum(all_recal==-1,axis=0))
            it_replaced.append(np.sum(all_recal==-2,axis=0))
            it_val.append(all_val)
            it_prev.append(all_prev)
            it_train.append(all_train)
            it_times.append(all_times)

            it_acc[it][:,:lda_ind] = it_acc[0][:,:lda_ind]
            it_recal[it][:lda_ind] = it_recal[0][:lda_ind]
            it_fail[it][:lda_ind] = it_fail[0][:lda_ind]
            it_replaced[it][:lda_ind] = it_replaced[0][:lda_ind]
            it_val[it][:,:lda_ind] = it_val[0][:,:lda_ind]
            it_prev[it][:,:lda_ind] = it_prev[0][:,:lda_ind]
            it_train[it][:,:lda_ind] = it_train[0][:,:lda_ind]
            it_times[it][:,:lda_ind] = it_times[0][:,:lda_ind]


        it_acc2 = cp.deepcopy(it_acc)
        for i in range(len(it_acc2)):
            x = it_val[i] < 0
            # print(x.type)
            # print(it_acc2[i].shape)
            # print(ave_val.shape)
            it_acc2[i][(it_acc[i]< 0) & (it_val[i] > 75)]= it_val[i][(it_acc[i]< 0)& (it_val[i] > 75)]
            # it_acc2[i][(it_acc[i]< 0)]= it_val[i][(it_acc[i]< 0)]


        ave_acc2 = np.nanmean(np.abs(np.array(it_acc2)),axis=0)
        ave_acc = np.nanmean(np.abs(np.array(it_acc)),axis=0)
        ave_val = np.nanmean(np.abs(np.array(it_val)),axis=0)
        ave_prev = np.nanmean(np.abs(np.array(it_prev)),axis=0)
        ave_train = np.nanmean(np.abs(np.array(it_train)),axis=0)
        ave_times = np.nanmean(np.abs(np.array(it_times)),axis=0)
        ave_recal = np.nanmean(np.array(it_recal),axis=0)
        ave_fail = np.nanmean(np.array(it_fail),axis=0)
        ave_replaced = np.nanmean(np.array(it_replaced),axis=0)

        std_acc2 = np.nanstd(np.abs(np.array(it_acc2)),axis=0)/np.sum(~np.isnan(np.array(it_acc2)),axis=0)
        std_acc = np.nanstd(np.abs(np.array(it_acc)),axis=0)/np.sum(~np.isnan(np.array(it_acc)),axis=0)
        std_val = np.nanstd(np.abs(np.array(it_val)),axis=0)/np.sum(~np.isnan(np.array(it_val)),axis=0)
        std_prev = np.nanstd(np.abs(np.array(it_prev)),axis=0)/np.sum(~np.isnan(np.array(it_prev)),axis=0)
        std_train = np.nanstd(np.abs(np.array(it_train)),axis=0)/np.sum(~np.isnan(np.array(it_train)),axis=0)
        std_times = np.nanstd(np.abs(np.array(it_times)),axis=0)/np.sum(~np.isnan(np.array(it_times)),axis=0)
        std_recal = np.nanstd(np.array(it_recal),axis=0)/np.sum(~np.isnan(np.array(it_recal)),axis=0)
        std_fail = np.nanstd(np.array(it_fail),axis=0)/np.sum(~np.isnan(np.array(it_fail)),axis=0)
        std_replaced = np.nanstd(np.array(it_replaced),axis=0)/np.sum(~np.isnan(np.array(it_replaced)),axis=0)

        fig, ax = plt.subplots(1,2,figsize=(20,5))
        for mod in plot_mod:
            plot_ind = mod_tot.index(mod)
            # ax.plot(ave_acc[2:,plot_ind],'.-',label= mod + ' = '+ str(ave_recal[plot_ind,0]) + ' +/- ' + str(std_recal[plot_ind,0]))
            x = ~np.isnan(ave_val[:,plot_ind]) 
            # ave_acc2 = cp.deepcopy(ave_acc)
            # ave_acc[x,plot_ind] = ave_val[x,plot_ind]

            ax[0].plot(ave_acc[:,plot_ind],'.-',ms=8,label= mod + ' = '+ "{:.2f}".format(ave_recal[plot_ind]) + ' +/- ' + "{:.2f}".format(std_recal[plot_ind]) + ', ' + "{:.2f}".format(ave_fail[plot_ind]) +' failed'+ ', ' + "{:.2f}".format(ave_replaced[plot_ind]) +' replaced')#str(std_recal[plot_ind,0]))
            # ax[0].plot(np.squeeze(np.where(x)),ave_acc[x,plot_ind],'kx',ms=12)
            ax[1].plot(ave_acc2[:,plot_ind],'.-',ms=8,label= mod + ' = '+ "{:.2f}".format(ave_recal[plot_ind]) + ' +/- ' + "{:.2f}".format(std_recal[plot_ind])+ ', ' + "{:.2f}".format(ave_fail[plot_ind]) + ' failed'+ ', ' + "{:.2f}".format(ave_replaced[plot_ind]) +' replaced')#+ str(std_recal[plot_ind,0]))
            # ax.plot(np.squeeze(np.where(x)), ave_val[~np.isnan(ave_val[:,plot_ind]),plot_ind],'.-',ms=8,label= mod + ' = '+ str(ave_recal[plot_ind,0]) + ' +/- ' + str(std_recal[plot_ind,0]))
            # plt.fill_between(np.arange(ave_acc[2:,plot_ind].shape[0]),ave_acc[2:,plot_ind]-std_acc[2:,plot_ind],ave_acc[2:,plot_ind]+std_acc[2:,plot_ind],alpha=.3)
        ax[1].legend()
        for i in range(2):
            ax[i].axhline(75, ls='--', color='grey')
            ax[i].set_ylim([0,100])


In [ ]:
import warnings
from sklearn.decomposition import PCA
# Count recalibrations
path = 'C:/Users/yteh/Documents/work/necal/home data/'
subs = os.listdir(path)
if 'skip' in subs:
    subs = np.delete(subs,subs.index('skip'))
bat = 32
load_mod = False
ft = 'tdar'
iter = 10
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')
test_mod = 0
with warnings.catch_warnings():
    warnings.simplefilter("ignore",category=RuntimeWarning)
    sub_b = []
    sub_s = []
    sub_sep=[]
    for sub in range(7):
        print(subs[sub])
        sub_path = path + subs[sub] + '/DATA/MAT/'
        all_files = os.listdir(sub_path)
        if 'skip' in all_files:
            all_files = np.delete(all_files,all_files.index('skip'))

        # load or initialize cnn weights
        acc = np.zeros((len(all_files),2))
        acc_val = np.zeros((len(all_files),2))
        acc_prev = np.zeros((len(all_files),2))
        acc_train = np.zeros((len(all_files),2))
        

        acc_i = 0

        # Loop through files
        for i in range(len(all_files)-1):              
            # load training file
            train_file = all_files[i]
            train_data, train_params = prd.load_caps_train(sub_path + train_file + '/traindata.mat')

            # load training file
            train_data, train_params, th = prd.threshold(train_data, train_params)
            val_data, val_params = train_data, train_params
                
            # get current dofs and create key
            if i > -1:
                
                if i == 0:
                    train_dof = np.unique(train_params[:,-1])
                    key = np.arange(len(train_dof))

                    n_dof = len(train_dof)

                    h = np.ones((len(all_files),n_dof))
                    b = np.ones((len(all_files),n_dof))
                    b2 = np.ones((len(all_files)))
                    sep = np.zeros((len(all_files),n_dof))
                    tot_b = np.ones((len(all_files,)))
                    h[:] = np.nan
                    b[:] = np.nan
                    b2[:] = np.nan
                    h2 = cp.deepcopy(b2)
                    cl_s = np.zeros((n_dof,))
                    cl_s2 = np.zeros((len(all_files),n_dof))
                
                else:
                    prev_ndof = [n_dof, key, train_dof]
                    # check if current dofs are all in old dof list
                    train_dof = np.unique(train_params[:,-1])
                            # print('cur: ' + str(train_dof))
                    key = np.zeros((len(train_dof),))
                    dof_ovlp = np.isin(train_dof,prev_ndof[2],assume_unique=True)
                    temp_dof = cp.deepcopy(train_dof)
                    # loop through dofs that are in previous dofs, set the keys
                    for dof in train_dof[dof_ovlp]:
                        key[train_dof==dof] = prev_ndof[1][prev_ndof[2]==dof]
                        temp_dof[train_dof==dof] = prev_ndof[2][prev_ndof[2]==dof]

                    # check if previous dofs has classes not in this set
                    dof_xtra = ~np.isin(prev_ndof[2],temp_dof,assume_unique=True)
                    temp_dof = np.hstack((temp_dof,prev_ndof[2][dof_xtra]))
                    key = np.hstack((key,prev_ndof[1][dof_xtra]))

                    # loop through dofs that are not in previous dofs (ie new classes), add keys
                    key_i = 1
                    xtra = False
                    for dof in train_dof[~dof_ovlp]:
                        print('removing train ' + str(dof))
                        ind = train_params[:,-1] == dof
                        train_params = train_params[~ind,...]
                        train_data = train_data[~ind,...]
                        ind = val_params[:,-1] == dof
                        val_params = val_params[~ind,...]
                        val_data = val_data[~ind,...]
                        key = np.delete(key,temp_dof==dof)
                        temp_dof = np.delete(temp_dof,temp_dof==dof)

                    train_dof = cp.deepcopy(temp_dof)
                
                key = key.astype(int)

                train_data, train_params = lp.check_labels(train_data,train_params,train_dof,key,False)
                # print(train_dof)

                _, x_clean_cnn, y_clean, _, x_train_cnn, y_train, x_train_lda, y_train_lda, emg_scale, scaler, _, _ = prd.prep_train_caps(train_data, train_params, prop_b=False, batch_size=bat, ft=ft, noise=False, split=False,num_classes=n_dof)

                skip = False
                for cl in range(len(train_dof)):
                    if np.sum(y_train_lda==key[cl]) == 0:
                        skip = True
                
                if not skip:
                    w, c, mu_class, _, v, N, cov_class = dlda.train_lda(x_train_lda, y_train_lda, key)
                x_train_lda = np.matmul(x_train_lda,v)
                # pca = PCA(n_components=5)
                # x_train_lda = pca.fit_transform(x_train_lda)

                
                # cl_s2[:] = np.nan
                # cl_s[:] = np.nan
                m_cl = np.zeros((n_dof,x_train_lda.shape[1]))
                s_cl = np.zeros((n_dof,x_train_lda.shape[1],x_train_lda.shape[1]))
                for cl in train_dof:
                    train_ind = np.squeeze(y_train_lda == key[train_dof==cl])
                    temp_x = x_train_lda[train_ind,...]
                    m_cl[key[train_dof==cl],...] = np.nanmean(temp_x,axis=0)
                    s_cl[key[train_dof==cl],...] = np.cov(temp_x.T)

            #del x_train_lda, y_train_lda, x_train_cnn, y_train, x_clean_cnn, y_clean
            
            # load data
            test_file = all_files[i+1]
            test_data, test_params = prd.load_caps_train(sub_path + test_file + '/traindata.mat')
            
            # check class labels
            test_data, test_params, _ = prd.threshold(test_data, test_params, th)
            test_data, test_params = lp.check_labels(test_data,test_params,train_dof,key)

            # test 
            y_test, _, x_test_cnn, x_test_lda, y_test_lda = prd.prep_test_caps(test_data, test_params, scaler, emg_scale, num_classes=n_dof, ft=ft, split=False)
            
            # test 
            acc[i+1,:] = lp.test_models(None, None,  x_test_lda, y_test_lda, lda=[w,c])
            x_test_lda = np.matmul(x_test_lda,v)
            # x_test_lda = pca.transform(x_test_lda)

            for cl in train_dof:
                test_ind = np.squeeze(y_test_lda == key[train_dof==cl])
                train_ind = np.squeeze(y_train_lda == key[train_dof==cl])
                m0 = np.nanmean(x_train_lda[train_ind,:],axis=0)
                s0 = np.cov(x_train_lda[train_ind,:].T)
                if np.sum(test_ind) > 10:
                    m2 = np.nanmean(x_test_lda[test_ind,:],axis=0)
                    s2 = np.cov(x_test_lda[test_ind,:].T)
                    h[i+1,key[train_dof==cl]] = prd.hellinger(m0,s0,m2,s2)
                    b[i+1,key[train_dof==cl]] = np.nanmean(prd.mahal(m0,s0,m2,s2))
                    ct = 0
                    for cl_i in train_dof:
                        cl_sep = np.nanmean(prd.mahal(np.squeeze(m_cl[key[train_dof==cl_i],...]),np.squeeze(s_cl[key[train_dof==cl_i],...]),m2,s2))
                        if np.isnan(cl_sep):
                            print('oops')
                        else:
                            ct += 1
                            sep[i+1,key[train_dof==cl]] += cl_sep
                    sep[i+1,key[train_dof==cl]] /= ct

                    ct = 0
                    for cl_i in train_dof:
                        if cl_i != cl:
                            temp_ind = np.squeeze(y_test_lda == key[train_dof==cl_i])
                            temp_x2 = x_test_lda[temp_ind,...]
                            m1 = np.nanmean(temp_x2,axis=0)
                            s1 = np.cov(temp_x2.T)
                            cl_s2[i+1,key[train_dof==cl]] += np.nanmean(prd.mahal(m1,s1,m2,s2))
                            ct+=1
                    cl_s2[i+1,key[train_dof==cl]] /= ct
                    # b[i+1,key[train_dof==cl]] = prd.bhatta(m1,s1,m2,s2)
            # print(h[i+1,:])
            m1 = np.nanmean(x_train_lda,axis=0)
            s1 = np.cov(x_train_lda.T)
            m2 = np.nanmean(x_test_lda,axis=0)
            s2 = np.cov(x_test_lda.T)
            # print(m1)
            # print(m2)
            b2[i+1] = np.nanmean(prd.mahal(m1,s1,m2,s2))
            h2[i+1] = prd.hellinger(m1,s1,m2,s2)
            # print(h2)

            print("{:.2f}".format(acc[i+1,0]))
            del y_test, x_test_cnn#, x_test_lda, y_test_lda, test_data, test_params

        print(np.median(acc,axis=0))
        with open(subs[sub] + '_acc.p','wb') as f:
            pickle.dump([acc,h,b,cl_s,sep,cl_s2,b2,h2],f)
        sub_b.append(b)
        sub_s.append(cl_s)
        sub_sep.append(sep)
        
        gc.collect(generation=2)

In [57]:
y1=h2_all
y2=acc_all
pearsonr(y1[~np.isnan(y1) & ~np.isnan(y2)& ~np.isinf(y1)& ~np.isinf(y2)],y2[~np.isnan(y1) & ~np.isnan(y2)& ~np.isinf(y1)& ~np.isinf(y2)])

(-0.8164381789126056, 1.3471597124045558e-43)

In [2]:
sns.set_palette("PuBu")
path = 'C:/Users/yteh/Documents/work/necal/home data/'
subs = os.listdir(path)
if 'skip' in subs:
    subs = np.delete(subs,subs.index('skip'))
fig,ax = plt.subplots(1,3,figsize=(15,5))
fig2,ax2 = plt.subplots()
med_acc = []
m = ['o','x','^','s','p','D','+']
m = ['o','o','x','^','^','s','s']
fi = ['none','full','full','none','full','none','full']
m = ['x']
for i in range(7):
    m.append('x')
# cmaps = ['Blues','Greens','BuPu','Oranges','Purples','Reds','PuRd']

with warnings.catch_warnings():
    warnings.simplefilter("ignore",category=RuntimeWarning)
    for sub in range(7):
        sub_path = path + subs[sub] + '/DATApre/MAT/'
        all_files = os.listdir(sub_path)
        if 'skip' in all_files:
            all_files = np.delete(all_files,all_files.index('skip'))
        # num_pre = len(all_files)

        with open('lda_hell/'+subs[sub] + '_hell2.p','rb') as f:
            acc,h,b,cl_s,sep,cl_s2,b2,h2 = pickle.load(f)
        
        if sub ==0:
            acc_all = acc[1:,0]
            h_all = np.nanmean(h[1:,:],axis=1)
            h2_all = h2[1:]
        else:
            acc_all = np.hstack((acc_all,acc[1:,0]))
            h_all = np.hstack((h_all,np.nanmean(h[1:,:],axis=1)))
            h2_all = np.hstack((h2_all,h2[1:,]))

        col = np.array(sns.color_palette("GnBu",acc.shape[0]-1))[:,:]
        # print(np.nanmedian(acc[1:num_pre,0]))
        # print(np.nanmedian(acc[num_pre:,0]))
        # med_acc.extend((np.nanmedian(acc[1:num_pre,0]),np.nanmedian(acc[num_pre:,0])))
        # print('---')
        min_h = np.nanmax(h[1:,:],axis=1)
        norm_m = b/np.nanmean(cl_s)
        acc_p = acc[1:,0]
        
        # fig1,ax1 = plt.subplots(1,3,figsize=(20,5))
        for c in range(acc_p.shape[0]):
            ax[0].plot(h2[c+1],acc_p[c], m[sub],mew=2,ms=9,label='TR' + str(sub),color=col[c,:],fillstyle=fi[sub])
            ax[1].plot(np.nanmean(h[c+1,:]),acc_p[c], m[sub],mew=2,ms=9,color=col[c,:],fillstyle=fi[sub])
            if c == 0:
                ax[2].plot(h2[c+1],np.nanmean(h[c+1,:]),m[sub],mew=2,ms=9,label='TR' + str(sub+1),color=col[c,:],fillstyle=fi[sub])
            else:
                # if c == acc_p.shape[0]-1:
                #     ax[2].legend()
                ax[2].plot(h2[c+1],np.nanmean(h[c+1,:]),m[sub],mew=2,ms=9,color=col[c,:],fillstyle=fi[sub])
        ax[2].set_ylim([0,1.1])
        ax[2].set_xlim([0,1.1])
        ax[0].set_ylim([0,100])
        ax[0].axhline(50, ls='--', color='grey')
        ax[1].set_ylim([0,100])
        ax[1].axhline(50, ls='--', color='grey')
        ax[0].set_ylabel('Accuracy (%)')
        ax[0].set_xlabel('Hellinger Distance P(X1) - P(X2)')
        ax[1].set_xlabel('Hellinger Distance P(X1|Y) - P(X2|Y)')
        ax[2].set_xlabel('Hellinger Distance P(X1) - P(X2)')
        ax[2].set_ylabel('Hellinger Distance P(X1|Y) - P(X2|Y)')
        for a in range(3):
            ax[a].set_xticks([0,.2,.4,.6,.8,1])
            

        # ax.set_xlim([-6,0])
        # ax.plot(b2[1:],acc_p, 'x')
        # ax1[0].plot(h2[1:], 'x',mew=2,ms=7)
        # ax1[1].plot(np.nanmean(h[1:,:],axis=1),  'x',mew=2,ms=7)
        # ax1[2].plot(acc_p, 'x',mew=2,ms=7)
        # ax1[2].set_ylim([0,100])
        # ax1[0].set_ylim([0,1])
        # ax1[1].set_ylim([0,1])
        


findfont: Font family ['Helvetica'] not found. Falling back to DejaVu Sans.


In [ ]:
temp = []
for sub in range(7):
    with open(subs[sub] + '_lda_accs.p','rb') as f:
        acc, _ = pickle.load(f)
    temp.append(acc[1,0])

## Test generation

In [ ]:
# Count recalibrations
path = 'C:/Users/yteh/Documents/work/necal/home data/'
subs = os.listdir(path)
if 'skip' in subs:
    subs = np.delete(subs,subs.index('skip'))
bat = 128
load_mod = False
mod_tot = ['blda','lda','crlda','alda','bcnn','cnn', 'crcnn','acnn03','acnn30','acewc00','acewc30', 'adcnn30', 'vcnn', 'bvcnn', 'avcnn03', 'avcnn15', 'acnnl03','crvcnn','acewclm','xtra','xtra1','xtra2']
ft = 'feat'
iter = 1

for sub in range(4,5):
    print(subs[sub])
    sub_path = path + subs[sub] + '/DATA/MAT/'
    all_files = os.listdir(sub_path)
    if 'skip' in all_files:
        all_files = np.delete(all_files,all_files.index('skip'))

    # first iteration, includes LDA; others exclude LDA
    mod_all = ['vcnn']

    # load or initialize cnn weights
    if load_mod:
        with open(subs[sub] + '_' + str(0) + '_r_accs.p','rb') as f:
            all_acc, all_recal, all_val, all_prev, all_train, all_times, _, _, c_weights, cl_wc, scaler_0, emg_scale = pickle.load(f)
        all_times = np.zeros((len(all_files),len(mod_tot)))
    else:
        c_weights = None
        v_weights = None
        v_wc = None
        cl_wc = None
        all_recal = np.empty((len(mod_tot),1))
        all_recal[:] = np.nan
        all_acc = np.zeros((len(all_files),len(mod_tot)))
        all_val = np.zeros((len(all_files),len(mod_tot)))
        all_prev = np.zeros((len(all_files),len(mod_tot)))
        all_train = np.zeros((len(all_files),len(mod_tot)))
        all_times = np.zeros((len(all_files),len(mod_tot)))

    mod_i = 0
    for mod in mod_all:
        acc = np.zeros((len(all_files),5))
        acc_val = np.zeros((len(all_files),5))
        acc_prev = np.zeros((len(all_files),5))
        acc_train = np.zeros((len(all_files),5))

        if 'cnn' in mod:
            acc_i = 2
        elif 'cewc' in mod:
            acc_i = 4
        elif 'lda' in mod:
            acc_i = 0

        cnn = None
        ewc = None

        ep = 50
        recal = 0
        skip = False

        # Loop through files
        for i in range(1,2):#len(all_files)-1):
            # load training file
            train_file = all_files[i]
            train_data, train_params = prd.load_caps_train(sub_path + train_file + '/traindata.mat')

            train_data, train_params, th = prd.threshold(train_data, train_params)
            val_data = train_data
            val_params = train_params

            train_dof = np.unique(train_params[:,-1])
            key = np.empty(train_dof.shape)
            for key_i in range(len(train_dof)):
                key[key_i] = cp.deepcopy(train_params[np.argmax(train_params[:,2] == train_dof[key_i]),0])
            n_dof = int(np.max(key))
            
            train_data, train_params = lp.check_labels(train_data,train_params,train_dof,key)
            val_data, val_params = lp.check_labels(val_data,val_params,train_dof,key)

            _, x_clean_cnn, y_clean, _, x_train_cnn, y_train, x_train_lda, y_train_lda, emg_scale, scaler, _, _, _ = prd.prep_train_caps(train_data, train_params, prop_b=False, batch_size=bat, ft=ft, noise=True, split=False,num_classes=n_dof)

            _, _, _, _, x_val_cnn, y_val, x_val_lda, y_val_lda, _, _, _, _, _ = prd.prep_train_caps(val_data, val_params, emg_scale=emg_scale,scaler=scaler, prop_b=False, batch_size=bat, ft=ft, num_classes=n_dof, noise=False, split=False)

            del train_data, train_params, val_data, val_params

            cnn, all_times[i,mod_tot.index(mod)] = lp.train_models(traincnn=x_train_cnn,y_train=y_train, mod=['vcnn'], n_dof=n_dof, ep=30, dec=True, print_b=True)
            cnn, all_times[i,mod_tot.index(mod)] = lp.train_models(traincnn=x_train_cnn,y_train=y_train, mod=[cnn], n_dof=n_dof, ep=30, dec=False,print_b=True)

In [ ]:
x_out = cp.deepcopy(x_clean_cnn)
for cl in range(y_clean.shape[1]):
    x_out[y_clean[:,cl]==1,...] = np.random.normal(np.mean(x_clean_cnn[y_clean[:,cl]==1,...],axis=0), np.std(x_clean_cnn[y_clean[:,cl]==1,...],axis=0),x_clean_cnn[y_clean[:,cl]==1,...].shape)


In [ ]:
x_out = cp.deepcopy(x_train_cnn)
for cl in range(y_train.shape[1]):
    x_out[y_train[:,cl]==1,...] = np.random.normal(np.mean(x_train_cnn[y_train[:,cl]==1,...],axis=0), np.std(x_train_cnn[y_train[:,cl]==1,...],axis=0),x_train_cnn[y_train[:,cl]==1,...].shape)


In [ ]:
samp1 = np.ones((1*y_clean.shape[0],8))
x_out,_,_ = cnn.dec(samp1,np.tile(np.argmax(y_clean,axis=1),[1]).astype('float32'),samp=True)
# _,x_out,_,_ = cnn(x_clean_cnn,np.tile(np.argmax(y_clean,axis=1),[1]).astype('float32'),dec=True)
x_out = x_out.numpy()
# for i in range(y_clean.shape[1]):
#     adjust1 = np.std(x_clean_cnn[np.argmax(y_clean,axis=1)==i,...],axis=0)
#     rescale = np.mean(adjust1)/np.mean(np.std(x_out[np.argmax(y_clean,axis=1)==i,...],axis=0))
#     gmean = np.mean(x_out[np.argmax(y_clean,axis=1)==i,...],axis=0)
#     x_out[np.argmax(y_clean,axis=1)==i,...] = (x_out[np.argmax(y_clean,axis=1)==i,...] - gmean)*rescale + gmean
# x_out = np.maximum(np.minimum(x_out,1),0)
for cl in range(y_clean.shape[1]):
    ind = np.tile(np.argmax(y_clean,axis=1)==cl,[1])
    ind2 = np.argmax(y_clean,axis=1)==cl
    x_temp = x_out[ind,...].reshape((np.sum(ind),-1))
    x_true = x_clean_cnn[ind2,...].reshape((np.sum(ind2),-1))
    # for i in range()
    plt.figure()
    # for i in range(x_true.shape[0]):
    #     plt.plot(x_true[i,...],'k-')
        
    for i in range(x_temp.shape[0]):
        plt.plot(x_temp[i,...],'b-')

In [ ]:
x_lda = x_clean_cnn.reshape(x_clean_cnn.shape[0],-1)
y_lda = np.argmax(y_clean,axis=1)[...,np.newaxis]
y_train_lda = np.argmax(y_clean,axis=1)[...,np.newaxis]
w,c, _, _, _, _, _ = dlda.train_lda(x_lda,y_lda)
y_out = dlda.predict(x_lda, w, c)
print(dlda.eval_lda(w, c, x_lda, y_lda))
x_out_lda = x_out.reshape(x_out.shape[0],-1)
print(dlda.eval_lda(w,c, x_out_lda,np.tile(y_train_lda,[1,1])))
w,c, _, _, _, _, _ = dlda.train_lda(x_out_lda,np.tile(y_train_lda,[1,1]))
print(dlda.eval_lda(w, c, x_out_lda,y_train_lda))
print(dlda.eval_lda(w, c, x_lda, np.argmax(y_clean,axis=1)[...,np.newaxis]))

In [ ]:
cnn1, _ = lp.train_models(traincnn=x_train_cnn,y_train=y_train, mod=['vcnn'], n_dof=n_dof, ep=15, dec=False,print_b=True)
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name="test_accuracy")
test_mod = dl.get_test(cnn1,test_accuracy)
print(lp.test_models(x_out, y_clean, None, None, cnn=cnn1, test_mod=test_mod, test_accuracy=test_accuracy))

cnn2, _ = lp.train_models(traincnn=x_out,y_train=y_clean, mod=['vcnn'], n_dof=n_dof, ep=15, dec=False,print_b=True)
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name="test_accuracy")
test_mod = dl.get_test(cnn2,test_accuracy)
print(lp.test_models(x_clean_cnn, y_clean, None, None, cnn=cnn2, test_mod=test_mod, test_accuracy=test_accuracy))

In [ ]:
cv_iter = 1
fig,ax = plt.subplots(1,5,figsize=(30,4))
for sub in range(2,3):#,5):
    with open(subs[sub] + '_0_r_accs.p','rb') as f:
        acc_all, recal_all, cur_all, prev_all, val_all, mod_all, mod_tot, c_weights, cl_wc, scaler_0, emg_scale= pickle.load(f)

    # mod_all = ['ld','bld','bcnnl','cnnl','acnnl','acnnl3','acnnl30','acewcl']
    colors =  cm.get_cmap('tab20c')
    c = np.empty((20,4))
    for i in range(20):
        c[i,:] = colors(i*1/20)


    nn_c = np.vstack((np.zeros((1,4)),c[8,:],c[0,:],c[9,:],c[1,:],c[10,:],c[2,:]))
    nn_c = np.vstack((np.zeros((1,4)),c[8,:],c[0,:],c[8,:],c[0,:],c[8,:],c[0,:],c[8,:],c[0,:],c[8,:],c[0,:]))
    nn_c[0,-1] = 1
    all_m = ['ko-','o-','o-','s-','s-','v-','v-']
    pt_m = ['ko-','o-','o-','o-','s','s','s','s','D']
    nn_c = np.vstack((np.array([0,0,0,1]), c[0,:],c[1,:],c[2,:],c[3,:],c[4,:],c[5,:],c[6,:],c[0,:],c[8,:],c[0,:]))
    # nn_c[0,-1] = 1

    labels = ['lda','mlp','cnn','a-mlp','a-cnn','ewc-mlp','ewc-cnn','c-mlp','c-cnn','c-ld']
    labels = ['r-lda','lda','cnn','r-cnn','a-cnn-5','a-cnn-3','a-cnn-30','ewc-cnn']
    # labels = mod_tot

    ax_ind = sub
    it = 0
    for v in [1,2]: 
        i = mod_tot.index(mod_all[v])
        acc_temp = acc_all[1:-1,i]
        if not np.isnan(acc_temp).all():
            x = np.arange(len(acc_temp))
            recal_i = (acc_temp < 0)
            ax[ax_ind].plot(np.abs(acc_temp),'-',color=nn_c[it,:])
            ax[ax_ind].plot(x[~recal_i],np.abs(acc_temp[~recal_i]),pt_m[it],label=labels[v],color=nn_c[it,:])
            ax[ax_ind].plot(x[recal_i],np.abs(acc_temp[recal_i]),'x',ms=10,color=nn_c[it,:])
            it+=1

    for i in range(5):
        ax[i].axhline(70, ls='--', color='grey')
        
        ax[i].set_ylim([0,100])
        ax[i].set_title('TR' + str(i+1))
    ax[0].legend()
    ax[2].set_xlabel('Calibration Set')
    ax[0].set_ylabel('Accuracy (%)')
    plt.rc('font', size=15) 

In [ ]:
cv_iter = 1
for sub in range(2,3):#,5):
    fig,ax = plt.subplots(1,4,figsize=(20,4))
    for it in range(0,cv_iter):
        with open(subs[sub] + '_' + str(it) + '_r_accs.p','rb') as f:
            # acc_all, recal_all = pickle.load(f)
            acc_all, recal_all, cur_all, prev_all, val_all,mod_all, mod_tot, c_weights, cl_wc, scaler_0, emg_scale= pickle.load(f)

        # mod_all = ['ld','bld','bcnnl','cnnl','acnnl','acnnl3','acnnl30','acewcl']
        colors =  cm.get_cmap('tab20c')
        c = np.empty((20,4))
        for i in range(20):
            c[i,:] = colors(i*1/20)


        nn_c = np.vstack((np.zeros((1,4)),c[8,:],c[0,:],c[9,:],c[1,:],c[10,:],c[2,:]))
        nn_c = np.vstack((np.zeros((1,4)),c[8,:],c[0,:],c[8,:],c[0,:],c[8,:],c[0,:],c[8,:],c[0,:],c[8,:],c[0,:]))
        nn_c[0,-1] = 1
        all_m = ['ko-','o-','o-','s-','s-','v-','v-']
        pt_m = ['ko','*','*','o','s','s','s','s','D']
        pt_m = ['ko','*','*','o','s','s','s','s','D']
        nn_c = np.vstack((np.array([0,0,0,1]),np.array([0,0,0,1]), c[0,:],c[1,:],c[4,:],c[5,:],c[6,:],c[8,:],c[6,:],c[0,:],c[8,:],c[0,:]))

        labels = ['lda','mlp','cnn','a-mlp','a-cnn','ewc-mlp','ewc-cnn','c-mlp','c-cnn','c-ld']
        labels = ['r-lda','lda','cnn','r-cnn','a-cnn-5','a-cnn-3','a-cnn-30','ewc-cnn']
        # labels = mod_tot

        ax_ind = [1,0,0,1,2,2,2,2,2,2,2,2,3,3,3,3]
        it = 0
        for v in range(len(mod_all)):
            i = mod_tot.index(mod_all[v])
            acc_temp = acc_all[1:-1,i]
            if not np.isnan(acc_temp).all():
                x = np.arange(len(acc_temp))
                recal_i = (acc_temp < 0)
                ax[ax_ind[it]].plot(np.abs(acc_temp),'-',color=nn_c[it,:])
                ax[ax_ind[it]].plot(x[~recal_i],np.abs(acc_temp[~recal_i]),pt_m[it],label=labels[it]+': ' + str(int(recal_all[i,0])),color=nn_c[it,:])
                ax[ax_ind[it]].plot(x[recal_i],np.abs(acc_temp[recal_i]),'x',ms=10,color=nn_c[it,:])
                it+=1

    for i in range(4):
        ax[i].axhline(70, ls='--', color='grey')
        ax[i].legend()
        ax[i].set_ylim([0,100])
    ax[0].set_ylabel('Accuracy (%)')

In [ ]:
from matplotlib.ticker import MaxNLocator
cv_iter = 1
for sub in range(0,5):
    fig,ax = plt.subplots(1,2,figsize=(15,5))
    for it in range(0,cv_iter):
        with open(subs[sub] + '_' + str(it) + '_r_accs.p','rb') as f:
            # acc_all, recal_all = pickle.load(f)
            acc_all, recal_all, cur_all, prev_all, mod_all, mod_tot, c_weights, cl_wc, scaler_0, emg_scale= pickle.load(f)

        # mod_all = ['ld','bld','bcnnl','cnnl','acnnl','acnnl3','acnnl30','acewcl']
        colors =  cm.get_cmap('tab20c')
        c = np.empty((20,4))
        for i in range(20):
            c[i,:] = colors(i*1/20)


        nn_c = np.vstack((np.zeros((1,4)),c[8,:],c[0,:],c[9,:],c[1,:],c[10,:],c[2,:]))
        nn_c = np.vstack((np.zeros((1,4)),c[8,:],c[0,:],c[8,:],c[0,:],c[8,:],c[0,:],c[8,:],c[0,:],c[8,:],c[0,:]))
        nn_c[0,-1] = 1
        all_m = ['ko-','o-','o-','s-','s-','v-','v-']
        pt_m = ['ko','o','*','o','s','D','s','s','D']
        nn_c = np.vstack((np.array([0,0,0,1]),np.array([0,0,0,1]), c[0,:],c[1,:],c[4,:],c[5,:],c[6,:],c[8,:],c[6,:],c[0,:],c[8,:],c[0,:]))

        labels = ['lda','mlp','cnn','a-mlp','a-cnn','ewc-mlp','ewc-cnn','c-mlp','c-cnn','c-ld']
        labels = ['r-lda','lda','cnn','r-cnn','a-cnn-5','a-cnn-3','a-cnn-30','ewc-cnn']
        labels = ['r-lda','lda','cnn','r-cnn','f-cnn-5','f-cnn-3','f-cnn-30','ewc-cnn']
        # labels = mod_tot

        ax_ind = [0,0,1,1,1,1,2,2,2,2,2,2,3,3,3,3]
        it = 0
        for v in [0, 3, 5, 4, 6, 7]: #range(len(mod_all)):
            i = mod_tot.index(mod_all[v])
            acc_temp = acc_all[1:-1,i]
            if not np.isnan(acc_temp).all():
                x = np.arange(len(acc_temp))
                recal_i = (acc_temp < 0)
                ax[ax_ind[it]].plot(np.abs(acc_temp),'-',color=nn_c[v,:])
                ax[ax_ind[it]].plot(x[~recal_i],np.abs(acc_temp[~recal_i]),pt_m[it],label=labels[v]+': ' + str(int(recal_all[i,0])),color=nn_c[v,:])
                ax[ax_ind[it]].plot(x[recal_i],np.abs(acc_temp[recal_i]),'x',ms=10,color=nn_c[v,:])
                ax[ax_ind[it]].xaxis.set_major_locator(MaxNLocator(integer=True))
                it+=1

    for i in range(2):
        ax[i].axhline(70, ls='--', color='grey')
        ax[i].legend()
        ax[i].set_ylim([0,100])
        

    ax[0].set_ylabel('Accuracy (%)')